# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [52]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import numpy as np
import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10]#, 42, 4] # don't change!
STRATEGIES = ['freeze']#, 'shuffle', 'homogeneous', 'heterogeneous'] # can be changed

In [4]:
### Training
def train(epoch, optimizer, criterion, dataloader):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(use_train=True)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, seed, dataloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        print("==> Saving to checkpoint..")
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 1 # number of iterations the model gets trained
learning_rates = { # learning rate is reset after specific epochs
    '1': 0.1,
    '150': 0.01,
    '250': 0.001
}
momentum = 0.9
weight_decay = 5e-4

start = timeit.default_timer()

print("Begin training..")
#Logging header

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
        print("\n==> Beginning training with seed {} and strategy {}".format(seed, strategy))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rates['1'], momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader()
        dataloader.download_cifar()
        dataloader.prepare_cifar(strategy, random_state=seed, batch_size=64)

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if str(epoch+1) in learning_rates.keys():
                print('==> Resetting learning rate')
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rates[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion, dataloader)
            test_acc, test_loss = test(epoch, test_acc, seed, dataloader)
            print("[{}/{}]: Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}"\
                  .format(epoch+1, num_epochs, train_acc, test_acc, train_loss, test_loss))
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
            
stop = timeit.default_timer()
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)
print(stop - start)

Begin training..
+------------------------------+
| Seeds: [[10]] |

==> Beginning training with seed 10 and strategy freeze
------------------------------
==> Building model..
==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Resetting learning rate
==> Saving to checkpoint..
[1/1]: Train Acc: 31.036 | Test Acc: 43.68 | Train Loss: 0.029438955476284025 | Test Loss: 0.023819254112243653
61.4310292


In [87]:
print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rates: {}".format(learning_rates), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)
print()

stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
log_separating_line(length_table)
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10]                                                                            | 90
| Strategies: ['freeze']                                                                 | 90
| -> Resulting number of iterations: 1                                                   | 90
| Number of iterations: 300                                                              | 90
| Learning rates: {'1': 0.1, '150': 0.01, '250': 0.001}                                  | 90
| Resuming from checkpoint: False                                                        | 90
+----------------------------------------------------------------------------------------+


+----------------------------------------------------------------------------------------+
Finished training. Time needed: 0 hrs 0 mins 0 secs


In [130]:
def log_separating_line(length_table):
    filling_chars = "-" * (length_table - 2)
    separating_line = "+{}+".format(filling_chars)
    print(separating_line)
    
def log_header_line(line, length_table, oc_char='|'):
    filling_ws = " " * (length_table - len(line) - 3)
    logged_line = "{} {}{}{}".format(oc_char, line, filling_ws, oc_char)
    print(logged_line)
    
def log_position_header(seed, strategy, length_table, oc_char='|'):
    filling_ws = ' ' * (length_table - 31)
    logged_line_1 = "{} Seed: {}    Strategy: {}{}{}".format(oc_char, str(seed).zfill(2), strategy, filling_ws, oc_char)
    filling_ws = ' ' * (length_table - 71)
    logged_line_2 = "{} Epoch         Train Accuracy  Test Accuracy   Train Loss   Test Loss{}{}"\
        .format(oc_char, filling_ws, oc_char)
    print(logged_line_1)
    print(logged_line_2)
    
def log_position_line(epoch, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table, oc_char='|'):
    epoch = str(epoch).zfill(len(str(num_epochs))) # leading zeros
    train_acc = str(np.round(train_acc, 2)).zfill(5)
    test_acc = str(np.round(test_acc, 2)).zfill(5)
    train_loss = str(np.round(train_loss, 5)).zfill(7)
    test_loss = str(np.round(test_loss, 5)).zfill(7)
    filling_ws = ' ' * (length_table - 69 )
    logged_line = "{} [{}/{}]:    {}           {}           {}      {}{}{}"\
        .format(oc_char, epoch, num_epochs, train_acc, test_acc, train_loss, test_loss, filling_ws, oc_char)
    print(logged_line)
    
log_separating_line(length_table)
log_position_header(seed, strategy, length_table)
log_separating_line(length_table)
log_position_line(epoch, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
log_separating_line(length_table)

+----------------------------------------------------------------------------------------+
| Seed: 10    Strategy: freeze                                                           |
| Epoch         Train Accuracy  Test Accuracy   Train Loss   Test Loss                   |
+----------------------------------------------------------------------------------------+
| [000/300]:    31.04           43.68           0.02944      0.02382                     |
+----------------------------------------------------------------------------------------+


--- 

# BackUp

resume = False
given_date = '20191113'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 300 # number of iterations the model gets trained
learning_rate = 0.1 # factor for weight updates
learning_rate_switches = { # learning rate is reset after specific epochs
    '150': 0.01,
    '250': 0.001
}
momentum = 0.9
weight_decay = 5e-4

start = timeit.default_timer()

rows = []
for seed in SEEDS:
    for shuffle_setting in [{'train': False, 'test': False}]:#, {'train': True, 'test': True}]:
        print("\n==> Beginning training with seed {} and shuffle setting {}".format(seed, shuffle_setting))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader()
        dataloader.download_cifar()
        dataloader.prepare_cifar('homogeneous', random_state=seed, batch_size=64)

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/ckpt_{}.pth'.format(given_date, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if str(epoch+1) in learning_rate_switches.keys():
                print('==> Resetting learning rate')
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate_switches[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion, dataloader)
            test_acc, test_loss = test(epoch, test_acc, seed, dataloader)
            print("[{}/{}]: Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}"\
                  .format(epoch+1, num_epochs, train_acc, test_acc, train_loss, test_loss))
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
            
stop = timeit.default_timer()
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'shuffle', 'accuracy', 'loss'])   
training_logs_dir = 'training_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)
